# LAB -- Quantum Key Distribution Laboratory

## Estimate the secret key rate you expect to see
Ultimately, I want to know the secure key rate (SKR) of the system, ie how many secure bits per second Alice and Bob can generate. *Calculation time for error correction and privacy amplification don't affect the SKR*

* How fast can Alice encode photons?
* What's the loss between Alice's source and Bob's measurement? If Alice sends 1 photon, what's the probability Bob gets a click?
* How good is Alice and Bob's equipment? ie if Alice sets her HWP to send a photon at $22.5^{\circ}$, how correct is that? If Bob thinks he's measuring a photon with polarization $22.5^{\circ}$, how often is he correct? 
* How much of the raw key gets lost through sifting, BER estimate, information reconciliation, and privacy amplification?


### Evaluate Equipment

1. Measure the efficiency of the APDs
 1. Calibrate attenuators
 2. Measure the power at a level the power meter can see
 3. Attenuate the known power and measure singles.
 4. Repeat measurement for all APDs.
 5. Put information in a table
2. Evaluate g2 to verify single photon source
 1. Why do we care?   
 1. See [HBT on QuTools](https://qutools.com/qued/qued-sample-experiments/sample-experiments-particle-nature-of-photons/)
1. The HWP
 1. How fast does it spin?
 2. How well does it rotate the polarization?
1. The PBS
 1. How well does it eliminate the other polarization?
 1. What is the loss?
 1. How much leaks through?
 1. Estimate how much that will contribute to loss and/or errors.

 

## Verify mean number of photons

### BB84's security is based on the no-cloning theorem
The no-cloning theorem states that for a given quanta in a superposition of eigenstates, it's impossible to produce a perfect copy. See, eg [Wootters and Zurek 1982]("https://www.nature.com/articles/299802a0") or [Dieks 1982]("https://www.sciencedirect.com/science/article/pii/0375960182900846?via%3Dihub").

Photon number splitting attack [Brassard, Lutkenhaus, Mor, Sanders 2000]("https://doi.org/10.1103/PhysRevLett.85.1330")

Need to have single photon source - how to verify that?

### Hanbury Brown Twiss
*There will need to be something about this in the "chapter" on QKD. It'd be good to talk about how g2 can indicate different types of light*
Need to measure the g2 of the source to verify it's single photon
 * Read the quTools-provided manual
 * Generate Python code to do the g2 measurement - need a code to do "if idler clicks, then measure signal on 2 detectors, take coincidence. Let students troubleshoot
 * A cool extension would be setting up HBT with variable time delay and then doing the measurements with coherent light, the heralded SPDC, and thermal light and showing the differences

In [1]:
# Pseudo code



## Generate Raw Key
1. Start with many photons per bit
1. Generate all the bases at the beginning

In [2]:
# Import necessary Libraries
# Generate the basis for each bit from Alice and Bob

import numpy as np
import random
import matplotlib.pyplot as plt

LENGTH = 10;

# Generate the random bases for Alice and Bob. Let 0 be HV and 1 be AD
basis_Alice = random.choices([0,1],k = LENGTH)
basis_Bob = random.choices([0,1],k = LENGTH)

# Now set the correct values for Alice and Bob's rotation
angles_Alice = np.zeros((LENGTH))
angles_Bob = np.zeros((LENGTH))
for x in range (LENGTH):
    angles_Bob[x] = 22.5*basis_Bob[x]
    angles_Alice[x] = 22.5*basis_Alice[x]
print(angles_Bob)
print(angles_Alice)

[22.5  0.   0.   0.  22.5 22.5  0.   0.  22.5  0. ]
[ 0.   0.   0.  22.5  0.  22.5 22.5  0.   0.   0. ]


In [3]:
# Place holder for actual data gathering

# Send in the correct angles for the polarizer

# Then read the data

# Repeat


## Sift the raw bits

You need to check several things:
   1. Did Bob get a click?
   1. If yes, did he have the same basis as Alice?

In [4]:
# Place holder for checking for Bob getting a click

# For each transmitted bit: did bob get any clicks? Select only the bits where Bob got a click

# If Bob got a click, tell us whether he got a 1 or a 0

# Then check if Bob and Alice have same basis. If yes, hooray, add that bit to the key


## Error Correction

I found a description of Cascade Error Correction [here](https://cascade-python.readthedocs.io/en/latest/protocol.html).

The below section uses a very simple one way Hamming code following the example in the section "Error Correction for Quantum Key Distribution" in Loepp and Wootters [book available online with MIT Library sign in](https://www.cambridge.org/core/books/abs/protecting-information/quantum-cryptography-revisited/8A88AA6C8C88E8A07E535FACEDE3C310).

In [5]:
# Estimate the q BER by revealing some of the bits.

length_reveal = 100

# errors = (bits_tx_sifted[0:length_reveal - 1] != bits_rx_sifted[0:length_reveal - 1]).astype(int)
# errors_total = errors.sum()
# BER = 1.0/length_reveal * errors.sum()

Al_all = np.random.randint(low = 0, high = 2, size = (length_reveal+30,))

# give Bob some errors for testing
err_vec = np.random.choice(2,length_reveal+30,p=[0.9,0.1])
Bob_all = Al_all ^ err_vec

key_test = Al_all[0:length_reveal]
Bob_test = Bob_all[0:length_reveal]
BERest =  np.mean(key_test != Bob_test)
print(BERest)

# Discard those bits
Al_all = Al_all[length_reveal:]
Bob_all = Bob_all[length_reveal:]





0.08


In [6]:
# Working out the Hamming Error matrix once

n = 7
k = np.log2(n+1).astype(int)

H = np.array([[1,1,1,0,1,0,0],
              [1,1,0,1,0,1,0],
              [0,1,1,1,0,0,1]])

# Define the error correction functions we'll need

def errCor(err_mat,H,a,b,n):
    # Multiply Alice's bits a by H
    at = np.matrix.transpose(a)
    Hat = np.matmul(H,at)
    Hat = np.remainder(Hat,2)

    # Multiply Bob's bits b by H
    bt = np.matrix.transpose(b)
    Hbt = np.matmul(H,bt)
    Hbt = np.remainder(Hbt,2)
    
    # Find Hb - Ha = s 
    s = Hat ^ Hbt
    sT = np.matrix.transpose(s)
    
    # s = He where e is the error vector.
    # If we know s, we can find which row of the error matrix it matches
    # Based on that, we know where the error is
    x = np.where((err_mat==sT).all(axis=1))[0][0].astype(int)
    err_vec = np.zeros((1,n))

    if x < n: 
     err_vec[0,x] = 1
     
    ev = err_vec.astype(int)
    ev = np.reshape(ev,(1,n))

    
    # Bob now has a corrected vector
    # bc = binSub(b,err_vec)
    bc = b ^ ev
    return bc

def makeErrMat(n,k,H):
    err_mat = np.zeros((n+1,k))

    for ii in range(n):
        err_vec = np.zeros((n,1))
        err_vec[ii] = 1
        HeT = np.matmul(H,err_vec)
        HeTT = np.matrix.transpose(HeT)
        # print('err loc',err_vec)
        # print('HeT', HeT)
        err_mat[ii] = HeTT    

    err_mat = err_mat.astype(int)
    return err_mat

err_mat = makeErrMat(n,k,H)

In [7]:
# do error correction for each block

# The following is an example script
LL = 31

Al_all = np.random.randint(low = 0, high = 2, size = (LL,))



err_all = np.zeros((LL)).astype(int)
err_all[3] = 1
err_all[26] = 1
err_all[18] = 1


Bob_all = Al_all ^ err_all


# We've skipped sifting for now
sk = Bob_all.size

cutoff =  sk % n # find key length modulo n

# elimate the "cut-off" bits
Bob = Bob_all[0:-cutoff]

# reshape Alice and Bob's bits into a 2d matrix
a = np.array([[1,0,0,0,1,1,1]])
b = np.array([[1,0,0,0,1,1,0]])


Bob = np.reshape(Bob,(-1,n))
Al  = np.reshape(Al_all[0:-cutoff],(-1,n))
Al[1,:] = a
Bob[1,:] = b

shape = np.shape(Al)
Bob_corr = np.zeros((shape)).astype(int) #initialize matrix for Bob's error correction
words = shape[0]

for ii in range(words):
    Al_word = Al[ii,:]
    Bob_word = Bob[ii,:]
    Bob_word_corr = errCor(err_mat, H, Al_word,Bob_word,n)
    Bob_corr[ii,:] = Bob_word_corr
    
error = np.mean(Al != Bob_corr)
print(error)


0.0


## Privacy Amplification

In this set-up, Eve isn't interacting with the optical set-up, so the only information she has is what Alice and Bob share via the classical channel. So, following pg 184 from the above book, we simply get rid of the last 3 bits of each of the 7-word chunks, and Eve will have no information about Alice and Bob's shared key.

In [8]:
# Take the giant matrix of error-correct keys, and just skip the last few columns (or rows)
Bob_corr = Bob_corr[:,0:4]
Al = Al[:,0:4]

# Then reshape the key into one long string of bits
Bob_key = np.reshape(Bob_corr,(1,-1))
Al_key = np.reshape(Al,(1,-1))

## With heralding
Now change the data taking so that you only measure clicks within a time $\Delta t$ of the herald.